In [1]:
import pandas
import mmengine
# info = pandas.read_pickle('data/kitti-360/kitti360_infos_all.pkl')

In [2]:
from projects.detr3d.custom_concat_dataset import CustomNusc
from projects.configs import debug
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend

In [3]:
register_all_modules()
cfg = copy.deepcopy(debug.nusc_val)
cfg['ann_file'] = 'nuscenes_infos_val.pkl'
cfg.pop('type')
cfg.pop('load_interval',None)
dataset = CustomNusc(**cfg)

04/13 13:37:50 - mmengine - INFO - ------------------------------
04/13 13:37:50 - mmengine - INFO - The length of the dataset: 6019
04/13 13:37:50 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| car                  | 80004  |
| truck                | 15704  |
| construction_vehicle | 2678   |
| bus                  | 3158   |
| trailer              | 4159   |
| motorcycle           | 2508   |
| bicycle              | 2381   |
| pedestrian           | 34347  |
+----------------------+--------+


In [5]:
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
import os.path as osp
import numpy as np
from projects.detr3d.detr3d import DETR3D
detr3d = DETR3D()
vis = visualizer_zlt(debug_name='',vis_count=10000)


In [5]:
tmp = dataset[930]

In [6]:
print(tmp['data_samples'].gt_instances_3d)

<InstanceData(

    META INFORMATION

    DATA FIELDS
    labels_3d: tensor([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
                0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1])
    bboxes_3d: LiDARInstance3DBoxes(
            tensor([[-11.8510, -30.6185,  -3.0888,   0.6470,   0.6210,   1.7780,  -1.5215],
                [  6.8793, -49.1384,  -2.8375,   0.9440,   0.6880,   1.9040,  -1.4650],
                [  6.5316, -38.0748,  -2.6415,   1.0300,   0.7510,   1.9750,  -1.5959],
                [  5.8789, -24.8877,  -2.6384,   0.6950,   0.6200,   1.5900,  -1.9291],
                [-20.6137, -23.4013,  -3.1468,   0.3310,   0.3450,   1.3260,   2.0177],
                [-21.2310,  14.3161,  -1.7814,   3.5540,   1.7320,   1.6830,  -2.9994],
                [  7.0661, -45.9239,  -2.8128,   0.6100,   0.6310,   1.9290,   1.7472],
                [-11.6270,   4.6788,  -1.9518,   0.5190,   0.5220,   1.3480,   1.2371],
                [-26.7955,  13.2351,  -1.8390,   3

In [7]:
tmp['data_samples'].metainfo['lidar_path']

'/localdata_ssd/nusc_dev1x/samples/LIDAR_TOP/n008-2018-08-01-15-16-36-0400__LIDAR_TOP__1533151611446636.pcd.bin'

In [6]:
import torch
sample_idx = '0a0d6b8c2e884134a3b48df43d54c36a'
trans = [ 646.2525, 1612.6713,    1.8371]# should be lidar2ego but not 2global
kpts = np.load('/home/zhenglt/mmdev11/detr3d-dev11/debug/0a0d6b8c2e884134a3b48df43d54c36a.npy')
kpts_3d = np.zeros((kpts.shape[0],7))
gt_labels_3d = torch.zeros((kpts.shape[0]))
kpts_3d[:,:2] = kpts
kpts_3d[:,2] -= trans[-1]
lidar_pts = np.load('/home/zhenglt/mmdev11/detr3d-dev11/debug/0a0d6b8c2e884134a3b48df43d54c36a.bin.npy')

In [7]:
from mmdet3d.structures import LiDARInstance3DBoxes
gt_bboxes_3d = LiDARInstance3DBoxes(torch.tensor(kpts_3d),box_dim=7,origin=(0.5,0.5,0.5))
map_inst = vis.toInstance({'gt_bboxes_3d': gt_bboxes_3d, 'gt_labels_3d': gt_labels_3d}, device=gt_labels_3d.device)
from mmengine.structures import InstanceData

/home/zhenglt/mmdev11/detr3d-dev11/projects/detr3d/vis_zlt.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(instances_3d['gt_labels_3d']).to(device)


In [8]:
for i in range(930,931,1):
    frame = dataset[i]
    name = osp.basename(frame['data_samples'].lidar_path).split('.')[0]
    scene = frame['data_samples'].city_name
    inst = frame['data_samples'].gt_instances_3d[:2]
    batch_data_samples = frame['data_samples']
    batch_input_metas = [batch_data_samples.metainfo]
    batch_input_metas = detr3d.add_lidar2img(batch_input_metas)
    gt_bboxes_3d = inst.bboxes_3d
    corners_pt = np.array(gt_bboxes_3d.corners.view(-1, 3)).T
    print(name)
    # vis.visualize(inst, batch_input_metas,None,'test','debug/')
    cat_instances = InstanceData.cat([map_inst, vis.add_score(frame['data_samples'].gt_instances_3d)])
    vis.visualize_dataset_item(frame,[cat_instances],pts = None,name_suffix='small_pcrange', dirname='debug/lyc_map')

n008-2018-08-01-15-16-36-0400__LIDAR_TOP__1533151611446636
reference torch.Size([406, 3])
900 1600
torch.Size([1, 405, 3])
frame: 930


In [10]:
# token2sample_idx
import pandas
import mmengine
infos = pandas.read_pickle('data/nus_v2/nuscenes_infos_trainval.pkl')



In [11]:
token2sampleidx = {}
for i,info in enumerate(infos['data_list']):
    token2sampleidx[info['token']] = i
mmengine.dump(token2sampleidx,'debug/lyc_map/token2idx_trainval.pkl')

{'e93e98b63d3b40209056d129dc53ceee': 0,
 '14d5adfe50bb4445bc3aa5fe607691a8': 1,
 'ae4e0c3aa3f24c91aab599e8b54e9264': 2,
 '8de7ec06e1ac48c689c4d24d6cc64fd7': 3,
 'ba94cb79ebc74614bc2442185cb53c26': 4,
 '4dfab70f0b3f405bbf17f7c534d2dd91': 5,
 '140c678882194ae69e47136c81168b6d': 6,
 'b8002d4effb540c0b1bb70addf423f32': 7,
 '1e19d0a5189b46f4b62aa47508f2983e': 8,
 'b7f4a3b32d654cc2b69c1c260a5f34a0': 9,
 '917a76cd5f564387bcacc22537010630': 10,
 'c0cd427afacb4c2bb974461eec1dbd79': 11,
 'c36eb85918a84a788e236f5c9eef2b05': 12,
 '5d054297a2bf43638a1d37cbb2ea2365': 13,
 'fcd37961086342abaf431243971cbc9b': 14,
 '6ec48adcac3b407ca77de01d2c600bf4': 15,
 '7ff1f84a3c184434a366282773ca9572': 16,
 '9253e18a84404e25b1eb4433905ad579': 17,
 '0037a705a2e04559b1bba6c01beca1cf': 18,
 '2acfc4b99f084778bc484fa1eef1b79f': 19,
 '5c95b81d9d404c219b4393da7e020280': 20,
 'e5bb15af564245e99b4def0019b2b045': 21,
 '2d0f28162f864ad391ddeefb4aaf5b05': 22,
 '026155aa1c554e2f87914ec9ba80acae': 23,
 'ec82914025214e10b88b887d